In [1]:
import pandas as pd
import os
import shutil
import yaml

In [2]:
db = os.environ.get('PSQL_DATABASE')
host = 'localhost'
user ='user'
port = '5432'
psql_engine = f'postgresql://{user}:@{host}:{port}/{db}'

In [3]:
backup_base_path = os.path.join('/media','trekker','Extreme SSD','PWGay_backup_12082023','public_html','pwgay','forum')
attachments_input_dir = os.path.join(backup_base_path, 'files')
attachments_output_dir = os.path.join('..', '_tests','assets', 'attachments')
if not os.path.exists(attachments_output_dir):
    os.makedirs(attachments_output_dir)

In [4]:
with open("get_attachment_list.sql", "r") as f:
    slq_statement = f.read()

In [5]:
df = pd.read_sql(slq_statement, psql_engine)

In [6]:
map_filenames = df[['physical_filename', 'real_filename']].to_dict(orient='records')

In [7]:
for file in map_filenames:
    old_filename = os.path.join(attachments_input_dir, file['physical_filename'])
    new_path = os.path.join(attachments_output_dir, file['physical_filename'])
    new_filename = os.path.join(new_path, file['real_filename'])
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    shutil.copy(old_filename, new_filename)

FileNotFoundError: [Errno 2] No such file or directory: '/media/trekker/Extreme SSD/PWGay_backup_12082023/public_html/pwgay/forum/files/25_f0bb71e1445b609a2807ee6246712cdc'

In [8]:
attachments_reg = {}
for i in df['post_msg_id'].unique():
    atts = df[df['post_msg_id'] == i]
    atts = atts[['att_order','physical_filename','real_filename','comment','ext','mimetype']]
    atts = atts.to_dict(orient='records')
    d = {item['att_order'] : {k : v for k, v in item.items() if k !='att_order'} for item in atts }
    attachments_reg[int(i)] = d

In [9]:
attachments_reg_path = os.path.join('..', '_tests', '_data', 'attachments.yml')
with open(attachments_reg_path, 'w') as f:
    f.write(yaml.dump(attachments_reg))